In [ ]:
%%capture
!pip install alpha_vantage
!pip install alpha_vantage pandas
!pip install pyalgotrading

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import pyalgotrading
import plotly.express as px  # (version 4.7.0)
import datetime
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.io as pio
from plotly.offline import plot
from datetime import datetime, timedelta, date
import math

In [ ]:
ts = TimeSeries(key='VGIB861XDLSFIRYP',output_format='pandas')

In [ ]:
class ScriptData:

  def __init__(self):
    self.__dict__ = dict()
    

  def __getitem__(self, script):
    return self.__dict__[script]

  def __contains__(self, script):
    return script in self.__dict__

  def fetch_intraday_data(self, script):

    # Fetches intraday data for given “script” (Example for script: “GOOGL”,“AAPL”) and stores as it is
    data, meta_data = ts.get_intraday(script)
    self.data, self.metadata = data, meta_data
    return data.iloc[::-1]

  def convert_intraday_data(self, script):
    
    data = self.data
    data.reset_index(inplace = True)
    data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    self.__dict__[script] = data


In [ ]:
def indicator1(df, timeperiod):


    df['indicator'] = df['close'].rolling(timeperiod).mean()
    dff = pd.DataFrame({'timestamp' : df['timestamp'], 'indicator' : df['indicator']})
    return dff


In [ ]:
class Strategy:

    script_data = ScriptData()

    def __init__(self, script):
      self.script = script
      
    def get_script_data(self):
      script_data.fetch_intraday_data(self.script)
      script_data.convert_intraday_data(self.script)
      return script_data[self.script]
    
    def get_signals(self):
      df1 = self.get_script_data()
      df2 = indicator1(script_data[self.script], timeperiod = 5)
      df2['close'] = df1['close']

      lst = ['NO_SIGNAL']

      for i in range(1, len(df2)):

        if (df2['close'][i - 1] > df2['indicator'][i - 1]) and (df2['close'][i] < df2['indicator'][i]):
          lst.append("SELL")

        elif (df2['close'][i - 1] < df2['indicator'][i - 1]) and (df2['close'][i] > df2['indicator'][i]):
          lst.append("BUY")

        else:
          lst.append("NO_SIGNAL")
        
      # print(len(lst))
      dff = pd.DataFrame({'timestamp' : df2['timestamp'], 'signal' : lst})

      dff = dff[dff['signal'] != 'NO_SIGNAL']
      dff.reset_index(inplace = True, drop = True)

      return dff
    
    

    

    

In [ ]:
script_data = ScriptData()
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,94.28,94.4000,94.2800,94.400,21749.0
1,2023-02-13 19:45:00,94.30,94.4700,94.2900,94.290,4524.0
2,2023-02-13 19:30:00,94.30,94.3900,94.2800,94.300,3308.0
3,2023-02-13 19:15:00,94.29,94.3000,94.2700,94.300,4382.0
4,2023-02-13 19:00:00,94.30,94.3000,94.2000,94.270,14263.0
...,...,...,...,...,...,...
95,2023-02-10 12:15:00,94.60,94.6513,94.2500,94.430,2322443.0
96,2023-02-10 12:00:00,94.99,95.1000,94.5800,94.605,1461925.0
97,2023-02-10 11:45:00,94.92,95.0100,94.6801,94.990,1397764.0
98,2023-02-10 11:30:00,95.19,95.3000,94.9100,94.920,1253697.0


In [ ]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,94.28,94.4000,94.2800,94.400,21749.0
1,2023-02-13 19:45:00,94.30,94.4700,94.2900,94.290,4524.0
2,2023-02-13 19:30:00,94.30,94.3900,94.2800,94.300,3308.0
3,2023-02-13 19:15:00,94.29,94.3000,94.2700,94.300,4382.0
4,2023-02-13 19:00:00,94.30,94.3000,94.2000,94.270,14263.0
...,...,...,...,...,...,...
95,2023-02-10 12:15:00,94.60,94.6513,94.2500,94.430,2322443.0
96,2023-02-10 12:00:00,94.99,95.1000,94.5800,94.605,1461925.0
97,2023-02-10 11:45:00,94.92,95.0100,94.6801,94.990,1397764.0
98,2023-02-10 11:30:00,95.19,95.3000,94.9100,94.920,1253697.0


In [ ]:
script_data.fetch_intraday_data("AAPL")
script_data.convert_intraday_data("AAPL")
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,153.65,153.7000,153.6000,153.7000,16098.0
1,2023-02-13 19:45:00,153.66,153.6700,153.6500,153.6700,16014.0
2,2023-02-13 19:30:00,153.65,153.6500,153.6101,153.6500,8276.0
3,2023-02-13 19:15:00,153.70,153.7000,153.6600,153.6700,12784.0
4,2023-02-13 19:00:00,153.70,153.7100,153.6800,153.7100,6443.0
...,...,...,...,...,...,...
95,2023-02-10 12:15:00,149.81,149.9000,149.5300,149.6500,1055904.0
96,2023-02-10 12:00:00,150.11,150.1600,149.7000,149.8200,1821615.0
97,2023-02-10 11:45:00,150.30,150.3499,150.0150,150.1100,1049982.0
98,2023-02-10 11:30:00,150.49,150.7100,150.2657,150.2854,1118830.0


In [ ]:
'GOOGL' in script_data

True

In [ ]:
'NVDA' in script_data

False

In [ ]:
df = indicator1(script_data['GOOGL'], timeperiod = 5)

In [ ]:
df

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:45:00,NaN
2,2023-02-13 19:30:00,NaN
3,2023-02-13 19:15:00,NaN
4,2023-02-13 19:00:00,94.312
...,...,...
95,2023-02-10 12:15:00,94.695
96,2023-02-10 12:00:00,94.626
97,2023-02-10 11:45:00,94.673
98,2023-02-10 11:30:00,94.743


In [ ]:
strategy = Strategy("NVDA")

In [ ]:
strategy.get_script_data()

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,218.310,218.38,218.2000,218.2000,14251.0
1,2023-02-13 19:45:00,218.200,218.36,218.1600,218.2600,17606.0
2,2023-02-13 19:30:00,218.150,218.34,218.1500,218.3000,4990.0
3,2023-02-13 19:15:00,218.200,218.37,218.1500,218.1500,7638.0
4,2023-02-13 19:00:00,218.130,218.36,218.0400,218.2000,5248.0
...,...,...,...,...,...,...
95,2023-02-10 12:15:00,210.190,210.55,208.9200,209.2400,2267725.0
96,2023-02-10 12:00:00,211.110,211.35,210.0800,210.2000,1713144.0
97,2023-02-10 11:45:00,212.639,212.67,210.8900,211.0800,2076220.0
98,2023-02-10 11:30:00,213.300,213.81,212.3663,212.6101,1507911.0


In [ ]:
strategy.get_signals()

,timestamp,signal
0,2023-02-13 18:30:00,BUY
1,2023-02-13 18:15:00,SELL
2,2023-02-13 17:45:00,BUY
3,2023-02-13 17:00:00,SELL
4,2023-02-13 16:30:00,BUY
5,2023-02-13 16:15:00,SELL
6,2023-02-13 15:15:00,BUY
7,2023-02-13 14:45:00,SELL
8,2023-02-13 14:15:00,BUY
9,2023-02-13 13:00:00,SELL


In [ ]:
df1 = strategy.get_script_data()
indicator1(df1, timeperiod = 5)
df1

,timestamp,open,high,low,close,volume,indicator
0,2023-02-13 20:00:00,218.310,218.38,218.2000,218.2000,14251.0,NaN
1,2023-02-13 19:45:00,218.200,218.36,218.1600,218.2600,17606.0,NaN
2,2023-02-13 19:30:00,218.150,218.34,218.1500,218.3000,4990.0,NaN
3,2023-02-13 19:15:00,218.200,218.37,218.1500,218.1500,7638.0,NaN
4,2023-02-13 19:00:00,218.130,218.36,218.0400,218.2000,5248.0,218.22200
...,...,...,...,...,...,...,...
95,2023-02-10 12:15:00,210.190,210.55,208.9200,209.2400,2267725.0,210.40572
96,2023-02-10 12:00:00,211.110,211.35,210.0800,210.2000,1713144.0,210.02372
97,2023-02-10 11:45:00,212.639,212.67,210.8900,211.0800,2076220.0,209.97770
98,2023-02-10 11:30:00,213.300,213.81,212.3663,212.6101,1507911.0,210.47502


In [ ]:
import numpy as np

In [ ]:
all_titles = ['SMA Crossover']
        
fig = make_subplots(rows=1, cols=1, subplot_titles=all_titles, specs=[[{"secondary_y": False}]],)

fig.add_trace(go.Scatter(x=df1.index, y=df1['indicator'],
                          mode='lines', line_width=1.5,
                          name='SMA', marker_color='Black'), secondary_y=False, row=1, col=1)

fig.add_trace(go.Candlestick(x=df1.index,
                          open=df1['open'],
                          high=df1['high'],
                          low=df1['low'],
                          close=df1['close'], 
                          showlegend=False, name='Historical Data'), secondary_y=False)

                 
fig.update_layout(hoverlabel_namelength=40)


fig.update_layout(xaxis = dict(
                              tickmode = 'array',
                              tickvals = np.arange(0, len(df1), 10),
                              ticktext = [df1['timestamp'][i] for i in np.arange(0, len(df1), 10)]
                              ),
                   font=dict(color="black"))

fig.update_layout(hovermode="x unified")
fig.update_yaxes(title_text="OHLC", secondary_y=False,showgrid=False,row=1,col=1)
fig.update_yaxes(showgrid=False)

fig.update_layout(width=1750,
                  height=875,
                  xaxis_title="Timeline",
                  yaxis_title="Price",)